In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

# Workspace
エージェント機能を持つチャットボットのサンプル。 mlflow と連携

In [0]:
import mlflow
import zoneinfo

tz_info = zoneinfo.ZoneInfo("Asia/Tokyo")
# mlflow.set_experiment("agent-rag")

# Playground

In [0]:
import datetime
import importlib
from mlflow.types.responses import Message, ResponsesAgentRequest

import agent

# importlib.reload(agent)

ymd = datetime.datetime.now(tz=tz_info).strftime("%Y%m%d_%H%M%S")
prompt = """
Databricks とその周辺の製品に関しての直近 (2025/06-) のニュースを振り返るため、まとめてください。
作業はタスクリストを作成し、タスクリストへチェックを付けながら step by step で処理していってください。

1. 対象製品名を検索しニュースを取得 (複数の呼び方へ対応する為、複数の検索ワードを試みてください)
2. 対象製品のニュースを月毎に整理
   - 対象製品自体だけでなく、関連するニュースも取得
   - 同月の他ニュースを検索し取得
3. 内容を整理し書出し
4. 書出したものを読み、指定期間を通した動向を複数の観点からまとめ、書出し
5. ここまで書き出したものを入力にレイアウトを整理した上で最終成果物として書出し
6. 最終成果物を読み、Output Requirement を満たすかを確認。満たせばその旨を表示。満たさない場合は 4番に戻り修正

<background>
読み手は ITの専門的知識を持つ人間を想定。指定期間のIT業界の動向を振り返る必要が有る。
また、業界の動向のみならず同時期に発生したイベントやニュースも見る事で、広い視野で業界の動向を知る
</background>

## Tool Guideline
- **search_news_history**: 製品に関する月毎のニュースを取得するのに使用。製品名や月 (yyyy/MM) を検索する事で検索可能

## Output Requirement
- 月単位で対象製品についてまとめてください
- 指定期間中の動向を複数の観点でまとめてください

例を以下に記載する。

```markdown
# yyyy/MM
## 製品名A
- (製品Aのニュース1。100文字前後)
- (製品Aのニュース2。100文字前後)

## 製品名B
- (製品Bのニュース3。100文字前後)
- (製品Bのニュース4。100文字前後)

## その他
- (同時期の他ニュース1。100文字前後)
- (同時期の他ニュース2。100文字前後)

# yyyy/MM
## 製品名A
- (製品Aのニュース5。100文字前後)
- (製品Aのニュース6。100文字前後)
```

"""
with mlflow.start_run(run_name=f"dev_{ymd}"):
    # 推論 (逐次)
    res = agent.agent_wrapped.predict(
        ResponsesAgentRequest(
            input=[
                Message(role="user", content=prompt),
            ]
        )
    )
    print(res.output)

In [0]:
print(res.output[-1].content[0]["text"])

# Test

In [0]:
import datetime
import importlib
import agent
import evaluate

# テストデータを作成
eval_dataset = [
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "東京都の明日の天気を教えてください。"}
            ]
        },
        "expectations": {"expected_response": "東京都の明日の天気は晴れです"},
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "横浜市の明日の天気を教えてください。"}
            ]
        },
        "expectations": {"expected_response": "横浜市の明日の天気は晴れです"},
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "群馬県の明日の天気を教えてください。"}
            ]
        },
        "expectations": {"expected_response": "群馬県の天気は豪雨です。"},
    },
]

ymd = datetime.datetime.now(tz=tz_info).strftime("%Y%m%d_%H%M%S")
with mlflow.start_run(run_name=f"dev_{ymd}"):
    # モデルを評価
    importlib.reload(agent)
    res = evaluate.eval_responses(agent.agent_wrapped, eval_dataset)
    print(res)